In [1]:
import numpy as np
import requests
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
#import os
#from dotenv import load_dotenv
#load_dotenv()
#API_KEY = os.getenv("API_KEY")

API_KEY = 'uEH5NtA'

request_url = f"http://strainapi.evanbusse.com/{API_KEY}/strains/search/all"
response = requests.get(request_url)
data = json.loads(response.text)

print(request_url)
print(type(response)) #> <class 'requests.models.Response'>
print(response.status_code) #> 200
print(type(response.text)) #> <class 'str'>
print(type(data)) #> <class 'dict'>

http://strainapi.evanbusse.com/uEH5NtA/strains/search/all
<class 'requests.models.Response'>
200
<class 'str'>
<class 'dict'>


In [2]:
# Get all strains
strain_names = pd.DataFrame(list(data.keys()), columns =['name'])

# Get strain characteristics
strain_values = list(data.values())
strain_values = pd.DataFrame.from_dict(strain_values)

# Add strain name to the dataframe
strains_raw = pd.concat([strain_names, strain_values], axis=1)

# Take a look at the data
strains_raw.head()

,name,id,race,flavors,effects
0,Afpak,1,hybrid,"[Earthy, Chemical, Pine]","{'positive': ['Relaxed', 'Hungry', 'Happy', 'S..."
1,African,2,sativa,"[Spicy/Herbal, Pungent, Earthy]","{'positive': ['Euphoric', 'Happy', 'Creative',..."
2,Afternoon Delight,3,hybrid,"[Pepper, Flowery, Pine]","{'positive': ['Relaxed', 'Hungry', 'Euphoric',..."
3,Afwreck,4,hybrid,"[Pine, Earthy, Flowery]","{'positive': ['Relaxed', 'Happy', 'Creative', ..."
4,Agent Orange,5,hybrid,"[Citrus, Orange, Sweet]","{'positive': ['Relaxed', 'Euphoric', 'Happy', ..."


In [3]:
# Get a line for each flavor
strains = strains_raw.flavors.apply(pd.Series) \
    .merge(strains_raw, right_index = True, left_index = True) \
    .drop(["flavors"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'effects'], value_name = "flavor") \
    .drop("variable", axis = 1) \
    .dropna()

# Get a line for each type of effect
strains = strains.effects.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["effects"], axis = 1)

# Get a line for each positive effect
strains = strains.positive.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["positive"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'negative', 'medical'], value_name = "positive_effect") \
    .drop("variable", axis = 1) \
    .dropna()

# Get a line for each negative effect
strains = strains.negative.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["negative"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'positive_effect', 'medical'], value_name = "negative_effect") \
    .drop("variable", axis = 1) \
    .dropna()

# Get a line for each medical effect
strains = strains.medical.apply(pd.Series) \
    .merge(strains, right_index = True, left_index = True) \
    .drop(["medical"], axis = 1) \
    .melt(id_vars = ['name', 'id', 'race', 'flavor', 'positive_effect', 'negative_effect'], \
          value_name = "medical_effect") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values(['id', 'flavor', 'positive_effect', 'negative_effect', 'medical_effect'], \
                 ascending = (1,1,1,1,1))

strains.head()

,name,id,race,flavor,positive_effect,negative_effect,medical_effect
10912,Afpak,1,hybrid,Chemical,Happy,Dizzy,Depression
77176,Afpak,1,hybrid,Chemical,Happy,Dizzy,Insomnia
275968,Afpak,1,hybrid,Chemical,Happy,Dizzy,Lack of Appetite
143440,Afpak,1,hybrid,Chemical,Happy,Dizzy,Pain
209704,Afpak,1,hybrid,Chemical,Happy,Dizzy,Stress


In [29]:
pip install plotly==4.5.2

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=b604a9602632a65a3be07cf7e513a8f8dc5931c78c8660b31154d995fd76b499
  Stored in directory: C:\Users\doina\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import plotly.graph_objects as go

#Cross-tabulate the example data frame.
df_cross = pd.crosstab(strains.race,strains.positive_effect)
# initiate data list for figure
data = []
#use for loop on every strain name to create bar data
for x in df_cross.columns:
   data.append(go.Bar(name=str(x), x=df_cross.index, y=df_cross[x]))

figure = go.Figure(data)
figure.update_layout(barmode = 'stack', title={
        'text': "Medical Mary Positive Effects",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
         xaxis_title="Strain Type",
                    yaxis_title="Negative Effects",
                    font=dict(
                    family="Courier New, monospace",
                    size=13,
                    color="#7f7f7f"))

#For you to take a look at the result use
figure.show()

In [31]:
#Cross-tabulate the example data frame.
df_cross = pd.crosstab(strains.race,strains.negative_effect)
# initiate data list for figure
data = []
#use for loop on every strain name to create bar data
for x in df_cross.columns:
   data.append(go.Bar(name=str(x), x=df_cross.index, y=df_cross[x]))

figure = go.Figure(data)
figure.update_layout(barmode = 'stack', title="Medical Mary Negative Effects",
                     xaxis_title="Strain Type",
                    yaxis_title="Negative Effects",
                    font=dict(
                    family="Courier New, monospace",
                    size=18,
                    color="#7f7f7f"))

#For you to take a look at the result use
figure.show()